In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

In [9]:
# MAIN 1
# Load Chrome driver and movement.uber.com/cities website
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

# Attributing the city name and the center-most zone code (or origin) to variables so they can be inserted in the URL later
city = 'boston' # Note: Atlanta might be missing data from 10/25/2018 - 10/29/2018
origin_code = '1074'
coordinates = '&lat.=33.7489&lng.=-84.4234622&z.=12'

# Open URL for the first day in the desired city (change coordinates depending on city)
driver.get('https://movement.uber.com/explore/' + city + '/travel-times/query?si' + origin_code + '&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=' + 
           '2016-01-02' + '&dt[dr][ed]=' + '2016-01-02' + '&cd=&sa;=&sdn=' + coordinates + '&lang=en-US')

In [10]:
# MAIN 2
# Defining the necessary funtions for the loop
def optOut():
    """
    Function that click on the opt out of some privacy settings.
    """
    try:
        priv_pref_buton = driver.find_element_by_id('privacy_pref_optin')
        priv_pref_buton.click()
        
    except:
        pass

    
def pressSkip():
    """
    Function that presses skip on the "Understanding origins" popup window.
    """
    try:
        time.sleep(5)
        skip_button = driver.find_element_by_css_selector('div.af.bk')
        skip_button.click()
            
    except:
        pass


def fillPersonalInfo():
    """"
    Function that fills in the personal information before finally downloading the csv file.
    """
    try:
        name = ''
        last_name = ''
        email = ''
        
        first_name = driver.find_element_by_id('firstName')
        first_name.send_keys(email)

        last_name = driver.find_element_by_id('lastName')
        last_name.send_keys(last_name)
                                       
        email = driver.find_element_by_id('email')
        email.send_keys(email)

        purpose_dropdown = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[3]/div/div[2]/div/div/div/form/div[3]/div/div/div/div[1]')
        purpose_dropdown.click()

        purpose = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div/div/div/div/ul/li[6]')
        purpose.click()

        opt_out_newsletter = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[3]/div/div[2]/div/div/div/form/div[5]/div/div/label[2]/div[2]')
        opt_out_newsletter.click()

    except:
        pass

# Agree to privacy preferences
optOut()
        
    
# Skip button only shows up the first time you open the Chrome browser
pressSkip()

                                    
# Choosing correct data parameters (Traffic Analysis Zone) and opening date bar in preparation for the calendar loop
# Zone type dropdown only shows up the first time you open the Chrome browser
zone_type_dropdown = WebDriverWait(driver, 8).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[3]/div/div[2]/div/div/div/div/div[1]/div[2]/div')))
zone_type_dropdown.click()

traffic_analysis_zones = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div/div/div/div/div/ul/li[2]')))
traffic_analysis_zones.click()
        

# Click on Download data
download_button = driver.find_element_by_css_selector('div.f5 button') # //button[contains(@data-baseweb, "button")]')
download_button.click()


# Download csv file
travel_times_download = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div[3]/div/div[2]/div/div[1]/div/button[1]')
travel_times_download.click()
time.sleep(1)

fillPersonalInfo()

# Download csv file
download_button2 = driver.find_element_by_xpath('//button[contains(@type, "submit")]')  # '//button[contains(@type, "submit")]')
download_button2.click()
time.sleep(2)

# Create URLs
def getURL():
    """"
    Function that creates one URL per date between the specified date range
    """
    date = datetime(2016,1,3)
    while date <= datetime(2020,3,31):
        yield ('https://movement.uber.com/explore/' + city + '/travel-times/query?si' + origin_code + '&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=' +
               date.strftime('%Y-%m-%d') + '&dt[dr][ed]=' + date.strftime('%Y-%m-%d') + '&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US')
        date += timedelta(days=1)


# Perform iteration through URLs downloading the datasets for each URL
i = 0
print('Number of generated URLs: ' + str(len(list(getURL()))))
for url in getURL():
    i += 1
    if i < 4:
        driver.execute_script("window.open('"+url+"', '_self')")
        print(url)
        
        # Agree to privacy preferences
        # optOut()
        
        # Press skip
        # pressSkip()
        
        # Click on "Download data"
        download_button = driver.find_element_by_css_selector('div.f5 button') # //button[contains(@data-baseweb, "button")]')
        download_button.click()
    
        # Download csv file
        download_button2 = driver.find_element_by_xpath('//button[contains(@type, "submit")]')  # '//button[contains(@type, "submit")]')
        download_button2.click()
        time.sleep(2)

Number of generated URLs: 1550
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2016-01-03&dt[dr][ed]=2016-01-03&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2016-01-04&dt[dr][ed]=2016-01-04&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US
https://movement.uber.com/explore/atlanta/travel-times/query?si1074&ti=&ag=taz&dt[tpb]=ALL_DAY&dt[wd;]=1,2,3,4,5,6,7&dt[dr][sd]=2016-01-05&dt[dr][ed]=2016-01-05&cd=&sa;=&sdn=&lat.=33.7489&lng.=-84.4234622&z.=12&lang=en-US


ElementClickInterceptedException: Message: element click intercepted: Element <button data-baseweb="button" class="by f6 ae ah f7 f8 f9 fa fb fc fd fe ff dj fg fh cx cy c6 fi fj fk bo bn bp bm b3 cb ck c0 fl fm fn fo fp fq fr fs ft fu fv fw fx fy f5 cd fz g0 g1">...</button> is not clickable at point (212, 634). Other element would receive the click: <div class="af i8 ah cd j2 ha i9 j3 j4 hn j5 j6 cy">...</div>
  (Session info: chrome=84.0.4147.105)


In [4]:
# Note: could make a list with all desred cities and for city in list, start with search and go back to main page